## First Step: create viewsheds based on 6 image locations
1. Load Data
    * DEMS
    * GPS csv file
2. Convert csv file to coordinates
3. Convert GPS coordinates (a feature layer) to shp
4. Create individual shps for each image point
    * Extract relevant values for names (names for new shapefiles)
    * Create 6 copies of the GOS shp (conatining all data) and delete all rows expt 1 point (then save shp)
5. Use Viewshed() to calculate viewshed rasters for each image


<i>Please Note this Notebook was completed within an ArcGIS Pro environment with access to Arcpy</i>

In [2]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *
from pathlib import Path

In [3]:
########################################## Loading DEM and GPS data ###########################################################
#(steps 1-4)
##relevant rasters needed for the viewshed...
DEM1 = Raster("D:/Fall_2022/Programming/Project/Data/30DEM/USGS_1_n49w114_20210607.tif")

#convert the csv table to points (run once)
arcpy.management.XYTableToPoint(r"C:\Users\runac\Downloads\Fall_2022\Programming\project_idea\JupNotebooks\GPS_metadata.csv", 
                                r"GPS_metadata_XYTableToPoint", 
                                "decimal_Long", 
                                "decimal_Lat",
                                "GPSAltitude", 
                                'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
###converting GPS data to a shp
arcpy.conversion.FeatureClassToShapefile("GPS_metadata_XYTableToPoint","D:\Fall_2022\Programming\Project\Viewshed_MT_clean\exported_Data")

##creating a list that stores the name of each image file (used later to create individual shp for each point)
List_shp = []
List_names = []
with arcpy.da.UpdateCursor("GPS_metadata_XYTableToPoint", ['Field1']) as cursor:
    for row in cursor:
        print(row[0])
        name = row[0].split('.')
        List_shp.append(f'{name[0]}.shp')
        List_names.append(row[0])
        
##copying featureclass and converting to individual shpfiles for each point
data = 'D:\Fall_2022\Programming\Project\Viewshed_MT_clean\exported_Data\GPS_metadata_XYTableToPoint.shp'
count = 0
for name in List_shp:
    #copying original file 
    arcpy.management.CopyFeatures(data, name)
    #creating shpfiles with 1 point by deleting all features excepts for 1 row
    with arcpy.da.UpdateCursor(name, ['Field1','decimal_La','decimal_Lo', 'GPSAltit_1']) as cursor:
        for row in cursor:
            if row[0] != List_names[count]:
                cursor.deleteRow()
    count += 1

image1.HEIC
image2.HEIC
image3.HEIC
image4.JPG
image5.JPG
image8.JPG


In [15]:
####################### Check to see if files are made ########################

                                    ########## DELETE LATER ###############
# iterate over specific files in 'exported_Data' to create and convert files with "raster_conversion()" 
files = Path('D:\Fall_2022\Programming\Project\Viewshed_MT_clean\exported_Data').glob('outvwshd0[1-9]')
for file in files:
    print(file)
    
### so the files are there, just can't find them with glob....

In [33]:
############### Creating a list of names to use for viewsheds, based on the number of images processed ###################
#(step 5)
List_Raster_V_names = []
List_outnames = []
number = 0
for files in range(len(List_shp)):
    if len(str(number)) < 2:
        str_num = "0" + str(number)
    else: 
        str_num = str(number)
    Name = "\outvwdshd" + str_num
    Variable = "\outViewshed" + str_num
    List_outnames.append(Name)
    List_Raster_V_names.append(Variable)
    number += 1

###################################################### VIEWSHED ############################################################
for shp, out_file, vwshd in zip(List_shp, List_outnames, List_Raster_V_names):
    # Execute Viewshed (DON'T NEED 'List_Raster_V_names')
    outViewshed = Viewshed(DEM1, shp, 1, "FLAT_EARTH", 0.13)
    # Save the output 
    path = "exported_Data" + out_file
    outViewshed.save(path)
    name = vwshd[1:len(vwshd)]
    print("saving", name)

    #exporting viewsheds as tifs NOT necessary 
    #need to make sure folders (PATH) is valid -- Code not excuting 
#     raster_out_path =  "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\exported_Data\View_Rasters" + vwshd + ".tif"
#     print(raster_out_path)
#     arcpy.CopyRaster_management("outViewshed", raster_out_path, "8_BIT_UNSIGNED")

['\\outvwdshd00', '\\outvwdshd01', '\\outvwdshd02', '\\outvwdshd03', '\\outvwdshd04', '\\outvwdshd05']
['\\outViewshed00', '\\outViewshed01', '\\outViewshed02', '\\outViewshed03', '\\outViewshed04', '\\outViewshed05']


## Next big step: extracting elevation values (maxes) in a 360 view around image points. 

1. Viewshed raster to Polygon
2. Create a minimum bounding circle on vectorized viewshed raster
3. Generate points on perimeter of circle (min bounding region)
4. Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile.
5. Use vectorized viewshed raster to mask DEM (resulting in a viewshed shaped DEM with real elevation values)
6. Extract the cell values from the clipped viewshed DEM with the Polylines. (HERE CURRENTLY!!)
    * For each line store the largest DEM value and save values in a table
7. Plot table

In [47]:
################################################## Viewshed raster to Polygon ########################################
##vectorizing resulting viewshed rasters

#list of names of the viewshed tifs
vwshd_tifs = ["outViewshed1", "outViewshed2", "outViewshed3", "outViewshed4", "outViewshed5", "outViewshed6"]
#list of names to be used for vectorized viewsheds
ras_to_polys = ["\Poly_outView1", "\Poly_outView2", "\Poly_outView3", "\Poly_outView4", "\Poly_outView5", "\Poly_outView6"]

####### Looping over "ras_to_polys" to convert ALL viewsheds to polygons
poly_vwshd_list = {}
number = 0
for tif, poly in zip(vwshd_tifs, ras_to_polys):
    number += 1
    #declaring location and name for vectorized viewsheds
    path = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb" + poly
    #will use these paths for the polygon viewsheds to update the viewsheds later
    name = "v" + str(number)
    poly_vwshd_list[name] = path
    #converting rasters to polygons
    arcpy.conversion.RasterToPolygon(tif, 
                                     path, 
                                     "SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

# #deleting polygons with gridcode = 0 (represents areas viewer cannot see)
for v in poly_vwshd_list.values(): 
    with arcpy.da.UpdateCursor(v, ['*']) as cursor:
        for row in cursor:
            if row[3] == 0:
                cursor.deleteRow()
    print("Saving & Updating:", v)

Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView1
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView2
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView3
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView4
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView5
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView8


In [56]:
########################### create a min bounding circle on vectorized viewshed raster #######################
# creating list of names for minimum bounding circles
path = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb"
min_bound_names = []
for number in range(len(ras_to_polys)): 
    circle_name = "\B_outView" + str(number + 1)
    min_bound_names.append(path + circle_name)

# creating minimum bounding circle around viewshds
for poly_vwshd, min_bound in zip(poly_vwshd_list.values(), min_bound_names):
    arcpy.management.MinimumBoundingGeometry(poly_vwshd, 
                                         min_bound, 
                                         "CIRCLE", 
                                         "ALL")

In [57]:
############################## Generate points on perimeter of circle (min bounding region) ######################
# creating list of names for points on minimum bouding circles
path = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb"
pts_names = []
for number in range(len(ras_to_polys)): 
    circle_name = "\Pts_View" + str(number + 1)
    pts_names.append(path + circle_name)

    # creating pts on minimum bounding circle
for circle, pts in zip(min_bound_names, pts_names):
    arcpy.management.GeneratePointsAlongLines(circle, 
                                          pts, 
                                          "PERCENTAGE",  
                                          Percentage=0.2, 
                                          Include_End_Points='END_POINTS')

##takes apporx 1 min to execute

In [83]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile ######
##create a def to be called later that extract the coordinates for each image.shp
def image_coords(path_to_image):
    '''
    Given the shapefile path you can extract the points coorndinates. 
    Returns a arcpy Point
    '''
    with arcpy.da.SearchCursor(path_to_image, ['*']) as cursor:
        for row in cursor:
            #assumption, shp is 1 point, thus only returning 1 point value, if shp with many pts provided will return last pt
            cp = arcpy.Point(row[1][0], row[1][1])
    return cp

def creating_polylines(filepath, anchor_point, connection_points): 
    """
    creates lines from each coordinate (image<#>.shp) to the points plotted on the minimum bounding circle 
    created around the viewshed. 
    filepath: insert desired name for each polyline file created with entire filepath (to .gdb)
    anchor_point: filepath to shapefile for each repective image's coordinates
    connection_points: filepath to the points created on the minimum bounding circle
    """
    # ##creating new file name and shapefile for polylines
    path = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb"
    new_fc = filepath
    name = filepath[(len(path)+1):len(filepath)]
    list_points = arcpy.Array([])
    arcpy.management.CreateFeatureclass(path, name, 'POLYLINE')


    ##Creating a line for each point on perimeter to corrsponding image (WORKS!)
    #looping through perimeter points
    with arcpy.da.SearchCursor(connection_points, ['*']) as cursor:
        for row in cursor:
            cp1 = image_coords(anchor_point) #pulling the same image point
            cp2 = arcpy.Point(row[1][0], row[1][1])
            list_points.append(cp2)
            list_points.append(cp1)
            #assignng spatial reference (should check to make sure its accurate)
            spatial_reference = arcpy.SpatialReference(4326)
            polyline = arcpy.Polyline(list_points, spatial_reference)
            #inserting new line to new feature class
            icur = arcpy.da.InsertCursor(new_fc, ["SHAPE@"])
            icur.insertRow([polyline])
            del icur

##CURRENT issue: 'unknown coord ref system' for polylines

In [84]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile ######

# creating paths and names for new polyline files 
path = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb"
polyline_names = []
for number in range(len(ras_to_polys)): 
    line_name = "\View_lines" + str(number + 1)
    polyline_names.append(path + line_name)

# polyline_names : recently declared above
# pts_names : calling this already made list of minimum bounding circle pts
# List_shp : calling this already made list of image locations 

#Running the function that'll create polylines for each image and corresponding viewshed points
for polyline, image, pts, in zip(polyline_names, List_shp, pts_names):
    print("Creating:", polyline[(len(path) + 1):len(polyline)])
    creating_polylines(polyline, image, pts)
    
#takes 1.5 mins to make lines when p = 0.2

Creating: View_lines1
Creating: View_lines2
Creating: View_lines3
Creating: View_lines4
Creating: View_lines5
Creating: View_lines6
